Převod SAPCE NA PCE

In [9]:
import copy # kopie PCE - abych nezasahoval do původního
import io   # nechci aby mi sapce vypisovalo do konzole
import contextlib

import numpy as np
import pandas as pd

from UQpy.distributions import Uniform, Normal, JointIndependent

import SAPCE

# Z sapce.pce udělá list pce objektů, každý má jeden výstup
def rozdel_sapce_na_pce(pce):
    C = np.asarray(pce.coefficients)  # koeficienty PCE a převedeny na numpy array, aby šly jednoduše brát po sloupcích
    

    # kdyby to bylo  1D, udělám (n,1)
    if C.ndim == 1:
        p1 = copy.deepcopy(pce)
        p1.coefficients = C.reshape(-1, 1)
        return [p1]

    # jinak (n baze, n vystupy)
    n_out = C.shape[1] # počet sloupců matice koeficientů
    pce_list = []

    for j in range(n_out):
        # kopie celého PCE
        p_j = copy.deepcopy(pce)
        p_j.coefficients = C[:, j].reshape(-1, 1) # m koeficienty v kopii PCE tak, aby měla jen tento výstup
        pce_list.append(p_j)
    return pce_list # vrátím seznam PCE objektů


# Zkouska funkčnosti funkce rozdel_sapce_na_pce: # 

# načtení dat - stejná jako budu používat pro THETA
data = pd.read_csv("Oakwood.csv", sep=";", header=0)
arr = data.to_numpy()

X = arr[:, :12]
Y_all = arr[:, 12:]

# 12 Vstupů
dist_Ec = Normal(13, 1)
dist_Relax = Uniform(30, 40)

dist_S1_Erel = Uniform(2, 1)
dist_S1_E50 = Normal(65, 5)
dist_S1_c = Normal(30, 5)
dist_S1_theta = Normal(30, 1)
dist_S1_ko = Uniform(0.6, 0.5)

dist_S2_Erel = Uniform(2, 1)
dist_S2_E50 = Normal(130, 10)
dist_S2_c = Normal(5, 1)
dist_S2_theta = Normal(42, 1)
dist_S2_ko = Uniform(0.45, 0.2)

marginals = [
    dist_Ec, dist_Relax,
    dist_S1_Erel, dist_S1_E50, dist_S1_c, dist_S1_theta, dist_S1_ko,
    dist_S2_Erel, dist_S2_E50, dist_S2_c, dist_S2_theta, dist_S2_ko
]
joint = JointIndependent(marginals=marginals)

# Predikce
rng = np.random.default_rng(123) #náhodnéí body 123 - pořád stejné
idx_all = np.arange(X.shape[0])

train_idx = rng.choice(idx_all, size=30, replace=False)
val_idx = np.array([i for i in idx_all if i not in set(train_idx)], dtype=int)  # validační indexy - všechny které nejsou tréninkové

X_train = X[train_idx, :]
Y_train = Y_all[train_idx, :]

X_val = X[val_idx, :]
Y_val = Y_all[val_idx, :]

# SAPCE
sapce = SAPCE.SensitivityAdaptivePCE(
    pdf=joint,
    exp_design_in=X_train,
    exp_design_out=Y_train,
    max_partial_degree=10,
    num_inputs=12,
)


silence = io.StringIO() 
with contextlib.redirect_stdout(silence), contextlib.redirect_stderr(silence): # Sapce nevypíše do konzole
    sapce.construct_adaptive_basis(max_condition_number=1e3) # podmíněnost
    sapce.construct_pruned_pce(cr=1e-8)  # redukce členů s malýp příspěvkem

pce_multi = sapce.pce
# rozdělení pce_multi = sapce.pce na list single PCE
pce_list = rozdel_sapce_na_pce(pce_multi)




print("původní tvar koeficientů:", np.asarray(pce_multi.coefficients).shape)
print("počet PCE v listu:", len(pce_list))
print("tvar koeficientů prvního PCE:", np.asarray(pce_list[0].coefficients).shape)
# predikce multi vs predikce z listu
Y_pred_multi = np.asarray(pce_multi.predict(X_val))
Y_pred_stack = np.column_stack([np.asarray(p.predict(X_val)).reshape(-1) for p in pce_list])
print("\nTvary predikcí:")
print("multi:", Y_pred_multi.shape)
print("stack:", Y_pred_stack.shape)

diff = np.abs(Y_pred_multi - Y_pred_stack)
print("\nRozdíl:")
print("max =", float(np.max(diff)))
print("mean =", float(np.mean(diff)))

print("\nPorovnání predikcí:")

print("\nPrvní 3 řádky a prvních 5 výstupů:")
print("SAPCE:\n", np.round(Y_pred_multi[:3, :5], 10))
print("PCE:\n", np.round(Y_pred_stack[:3, :5], 10))

původní tvar koeficientů: (28, 23)
počet PCE v listu: 23
tvar koeficientů prvního PCE: (28, 1)

Tvary predikcí:
multi: (970, 23)
stack: (970, 23)

Rozdíl:
max = 1.3877787807814457e-17
mean = 1.0465804528707913e-18

Porovnání predikcí:

První 3 řádky a prvních 5 výstupů:
SAPCE:
 [[ 0.          0.00236809 -0.00322555 -0.00888042 -0.00381383]
 [ 0.          0.00271704 -0.00381716 -0.01013221 -0.00582512]
 [ 0.          0.00211005 -0.00224409 -0.00849923 -0.00362641]]
PCE:
 [[ 0.          0.00236809 -0.00322555 -0.00888042 -0.00381383]
 [ 0.          0.00271704 -0.00381716 -0.01013221 -0.00582512]
 [ 0.          0.00211005 -0.00224409 -0.00849923 -0.00362641]]
